In [0]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from os import path
import re
import csv
import pickle
#import nltk
#from statistics import *
#nltk.download("popular")

In [0]:
import import_ipynb
from external.model_train import model_data
from external.model_train import load_model
from external.model_train import inference_model
from external.model_train import predict_beamsearch
from external.model_train import evaluate
from external.model_train import clean_sent
from external.model_train import model_sent_data
from external.model_train import ppl_2
from external.model_train import ppl_e
from external.model_train import random_model

importing Jupyter notebook from /content/external/model_train.ipynb


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
!pip install import-ipynb
!pip install PrettyTable

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def final(weight_path,file_path = None, name = None, sent = None, print_output_only = False, no_of_output_print = None):
  '''
  final function which get raw data and return preformance matrix and print output prediction
  '''
  if sent == None:
    #load raw data from file and convert them into model ready data
    encoder_input_data = None
    decoder_input_data = None
    decoder_target_data = None

    encoder_input_data, decoder_input_data, decoder_target_data = model_data(df_name = name, file_path = file_path)

    #load best model
    new_model = load_model(weight_path = weight_path)

    #evaluate on data
    loss = None
    accuracy = None
    ppl_e = None
    ppl_2 = None
    loss, accuracy, ppl_e, ppl_2 = evaluate(new_model, encoder_input_data, decoder_input_data, decoder_target_data)

    if print_output_only == True and no_of_output_print != None:
      #load inference model
      inf_encoder_model = None
      inf_decoder_model = None
      inf_encoder_model,inf_decoder_model =  inference_model(new_model)

      decoded_sent=[]
      df =None
      df = pd.read_csv(file_path)
      for seq_index in (range(no_of_output_print + 5)):
        seq_index = seq_index + 5
        input_seq = encoder_input_data[seq_index: seq_index+1]
        decoded_sentence = predict_beamsearch(input_seq,inf_encoder_model,inf_decoder_model)
        print('--------------------------------------')
        print('Input sentence:', df['X'][seq_index])
        print('Decoded sentence:', decoded_sentence)
      encoder_input_data = None
      decoder_input_data = None
      decoder_target_data = None
      df = None
      del(encoder_input_data,decoder_input_data,decoder_target_data,df)
      gc.collect()
      return(loss, accuracy, ppl_e, ppl_2)
    else:
      encoder_input_data = None
      decoder_input_data = None
      decoder_target_data = None
      df =None
      del(encoder_input_data,decoder_input_data,decoder_target_data,df)
      gc.collect()
      return(loss, accuracy, ppl_e, ppl_2)
  else:
    cleaned_sent = clean_sent(sent)
    encoder_input_data = None
    encoder_input_data = model_sent_data(cleaned_sent)

    #load best model
    new_model = load_model(weight_path = weight_path)

    #load inference model
    inf_encoder_model = None
    inf_decoder_model = None
    inf_encoder_model,inf_decoder_model =  inference_model(new_model)
    input_seq = encoder_input_data
    decoded_sentence = predict_beamsearch(input_seq,inf_encoder_model,inf_decoder_model)
    #print('--------------------------------------')
    #print('Input sentence:', cleaned_sent)
    #print('Decoded sentence:', decoded_sentence)
    return(decoded_sentence)


In [0]:
#file path to data
dirName = '/content/drive/My Drive/char_level_attention_model'

train_path = dirName + '/shuffled_train.csv'
test_path = dirName + '/shuffled_test.csv'
cv_path = dirName + '/shuffled_cv.csv'
weight_path = dirName + '/weights/saved-model-146-0.9879.pkl'


# <font color=#0066cc> Random Model  </font>


In [0]:
random_loss,ranodm_ppl_e,random_ppl_2 = random_model(file_path = train_path)

100%|██████████| 206995/206995 [00:07<00:00, 26321.22it/s]
206995it [00:03, 60367.29it/s]


encoder_input_data_train(206995, 60, 37)
decoder_input_data_train(206995, 65, 37)
decoder_target_data_train(206995, 65, 37)


In [0]:
print("train_loss_on_radom_model",random_loss)
print("train_ppl_e_on_radom_model",ranodm_ppl_e)
print("train_ppl_2_on_radom_model",random_ppl_2)

train_loss_on_radom_model 6.1422033
train_ppl_e_on_radom_model 465.07715
train_ppl_2_on_radom_model 70.629715


# <font color=#0066cc> Evaluating model on Train_data  </font>


In [0]:
train_loss, train_accuracy, train_ppl_e, train_ppl_2 = final(weight_path= weight_path,file_path = train_path,name= "train",print_output_only = True,no_of_output_print = 100)

100%|██████████| 206995/206995 [00:07<00:00, 26034.57it/s]
206995it [00:03, 60065.03it/s]


encoder_input_data_train(206995, 60, 37)
decoder_input_data_train(206995, 65, 37)
decoder_target_data_train(206995, 65, 37)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
206995/206995 [==============================] - 49s 239us/sample - loss: 0.0428 - categorical_accuracy: 0.9880 - ppl_e: 1.0363 - ppl_2: 1.0250
--------------------------------------
Input sentence: ski trip last february, first come first serve
Decoded sentence: 	 d. 

--------------------------------------
Input sentence: barr
Decoded sentence

In [0]:
encoder_input_data = None
decoder_input_data = None
decoder_target_data = None
del(encoder_input_data,decoder_input_data,decoder_target_data)
gc.collect()

382

# <font color=#0066cc> Evaluating model on Cv_data  </font>


In [0]:
cv_loss, cv_accuracy, cv_ppl_e, cv_ppl_2 = final(weight_path = weight_path ,file_path = cv_path,name = "cv",print_output_only = True,no_of_output_print = 100)

100%|██████████| 50945/50945 [00:01<00:00, 26020.84it/s]
50945it [00:00, 60172.96it/s]


encoder_input_data_cv(50945, 60, 37)
decoder_input_data_cv(50945, 65, 37)
decoder_target_data_cv(50945, 65, 37)
50945/50945 [==============================] - 11s 218us/sample - loss: 0.0431 - categorical_accuracy: 0.9879 - ppl_e: 1.0366 - ppl_2: 1.0252
--------------------------------------
Input sentence: meeting to your manage
Decoded sentence: 	 rs. 

--------------------------------------
Input sentence:  kevin, here's the file for your forecast
Decoded sentence: 	 .    

--------------------------------------
Input sentence:  why does everyb
Decoded sentence: 	 ody in this know my p l????? 

--------------------------------------
Input sentence: picking up cl
Decoded sentence: 	 eveland 

--------------------------------------
Input sentence: please review and
Decoded sentence: 	  print 

--------------------------------------
Input sentence: "b" but haven'
Decoded sentence: 	 t seen anything on paper. 

--------------------------------------
Input sentence:  just chec
Decoded se

/content/external/model_train.ipynb:53: RuntimeWarning: divide by zero encountered in log
  },


--------------------------------------
Input sentence: letter and comment before
Decoded sentence: 	  it goes out? 

--------------------------------------
Input sentence:  do you know any phone num
Decoded sentence: 	 ber for bmg? 

--------------------------------------
Input sentence: pas
Decoded sentence: 	 s it back to me for the file. thanks. 

--------------------------------------
Input sentence: the
Decoded sentence: 	  attachment! here it is........ 

--------------------------------------
Input sentence: as p
Decoded sentence: 	 romised. not randy's 

--------------------------------------
Input sentence: tha
Decoded sentence: 	 nks for offering your previous memo. 

--------------------------------------
Input sentence:  with charley of richards, la
Decoded sentence: 	 ve tankent 

--------------------------------------
Input sentence: to and the draft incl
Decoded sentence: 	 uda me daing 

--------------------------------------
Input sentence: mucus every two weeks otherw

In [0]:
encoder_input_data = None
decoder_input_data = None
decoder_target_data = None
del(encoder_input_data,decoder_input_data,decoder_target_data)
gc.collect()

382

# <font color=#0066cc> Evaluating model on Test_data  </font>


In [0]:
test_loss, test_accuracy, test_ppl_e, test_ppl_2 = final(weight_path= weight_path,file_path= test_path,name= "test",print_output_only = True,no_of_output_print = 100)

100%|██████████| 51463/51463 [00:02<00:00, 25635.49it/s]
51463it [00:00, 60267.98it/s]


encoder_input_data_test(51463, 60, 37)
decoder_input_data_test(51463, 65, 37)
decoder_target_data_test(51463, 65, 37)
51463/51463 [==============================] - 14s 272us/sample - loss: 1.4165 - categorical_accuracy: 0.8489 - ppl_e: 4.1319 - ppl_2: 2.6685
--------------------------------------
Input sentence: please see attached memo a
Decoded sentence: 	 nd 

--------------------------------------
Input sentence:  let me know
Decoded sentence: 	  if you have any freetime 

--------------------------------------
Input sentence:  i can't find you guys over here
Decoded sentence: 	  'o it. 

--------------------------------------
Input sentence: note from regarding oman
Decoded sentence: 	 iees. 

--------------------------------------
Input sentence:  gents, thanks for your h
Decoded sentence: 	 elp  

--------------------------------------
Input sentence: these counterparties by the next legal/credit meet
Decoded sentence: 	 ing 

--------------------------------------
Input senten

In [0]:
encoder_input_data = None
decoder_input_data = None
decoder_target_data = None
del(encoder_input_data,decoder_input_data,decoder_target_data)
gc.collect()

382

# <font color=#0066cc> Prediction on Sentence  </font>


In [0]:
sent_list = [
    'hi there',
    'hell',
    'presentation please fin',
    'resignation please find at',
    'resignation please ',
    'let me ',
    'please rev',
    'please ca',
    'thanks fo',
    'Let me kno',
    'Let me know if y',
]

In [0]:
for sent in sent_list:
  pred_sent = final(sent=sent,weight_path=weight_path)
  print("\n")
  print(sent)
  print(pred_sent)
  print("="*50)

1it [00:00, 2519.10it/s]
1it [00:00, 4614.20it/s]



hi there
	 . we need to discuss 




1it [00:00, 3782.06it/s]



hell
	 o, please e mail your to 




1it [00:00, 4675.92it/s]

...


1it [00:00, 4060.31it/s]



thanks fo
	 r fowarding my resume. let me 




1it [00:00, 5356.71it/s]



Let me kno
	 w if you need anything else. 





Let me know if y
	 ou get this. outlook is up 



# <font color=#0066cc> Results  </font>


# <font color=#0066cc> Random Model  </font>


In [0]:
from prettytable import PrettyTable
table = PrettyTable(field_names=['Data_set','Loss','Perplexity_2','Perplexity_e'])
table.add_row(['Train_data',random_loss,ranodm_ppl_e,random_ppl_2])
print(table)

+------------+-----------+--------------+--------------+
|  Data_set  |    Loss   | Perplexity_2 | Perplexity_e |
+------------+-----------+--------------+--------------+
| Train_data | 6.1422033 |  465.07715   |  70.629715   |
+------------+-----------+--------------+--------------+


# <font color=#0066cc> Encoder-Decoder_Attention model  </font>


In [0]:
from prettytable import PrettyTable
table = PrettyTable(field_names=['Data_set','Loss','Categorical_Accuracy','Perplexity_2','Perplexity_e'])
table.add_row(['Train_data',train_loss, train_accuracy, train_ppl_e, train_ppl_2])
table.add_row(['CV_data',cv_loss, cv_accuracy, cv_ppl_e, cv_ppl_2])
table.add_row(['Test_data',test_loss, test_accuracy, test_ppl_e, test_ppl_2])
print(table)

+------------+---------------------+----------------------+--------------+--------------+
|  Data_set  |         Loss        | Categorical_Accuracy | Perplexity_2 | Perplexity_e |
+------------+---------------------+----------------------+--------------+--------------+
| Train_data |  0.0427812962884396 |      0.9879876       |  1.0363204   |   1.025035   |
|  CV_data   | 0.04309515465547812 |       0.987936       |  1.0366216   |  1.0252416   |
| Test_data  |  1.4164843144906127 |      0.8489191       |   4.131907   |  2.6684885   |
+------------+---------------------+----------------------+--------------+--------------+
